In [18]:
import os
import json
import keras_ocr
import cv2
import numpy as np
import tensorflow as tf

In [3]:
# Путь к папке с изображениями и аннотациями
data_dir = 'images'
annotations_file = os.path.join(data_dir, 'annotations.json')

In [4]:
# Загрузка аннотаций из файла
with open(annotations_file, 'r', encoding='utf-8') as f:
    annotations = json.load(f)

In [5]:
# Настраиваем алфавит для распознавания якутского текста
alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюяҕҥөһү '

In [6]:
# Создаем пайплайн Keras-OCR и настраиваем распознаватель
recognizer = keras_ocr.recognition.Recognizer(alphabet=alphabet)

2024-09-27 03:17:41.929629: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-27 03:17:44.483180: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 19510 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:01:00.0, compute capability: 8.6
2024-09-27 03:17:44.484000: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 16858 MB memory:  -> device: 1, name: NVIDIA RTX A5000, pci bus id: 0000:24:00.0, compute capability: 8.6
2024-09-27 03:17:44.484677: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:2 w

Provided alphabet does not match pretrained alphabet. Using backbone weights only.
Looking for /root/.keras-ocr/crnn_kurapan_notop.h5


In [7]:
# Функция для загрузки изображений с помощью OpenCV
def load_images_and_annotations(annotations):
    images = []
    texts = []
    for annotation in annotations:
        image_path = annotation['image']
        text = annotation['text']
        
        # Загружаем изображение с помощью OpenCV
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError(f"Не удалось загрузить изображение: {image_path}")
        
        # Конвертация изображения в формат RGB
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        images.append(img_rgb)
        texts.append(text)
    
    return images, texts

In [8]:
# Загружаем изображения и тексты из аннотаций
images, texts = load_images_and_annotations(annotations)

In [9]:
# Разделяем данные на тренировочные и тестовые
split_idx = int(len(images) * 0.8)
train_images, test_images = images[:split_idx], images[split_idx:]
train_texts, test_texts = texts[:split_idx], texts[split_idx:]

In [10]:
train_generator = recognizer.get_batch_generator(
    image_generator=zip(train_images, train_texts),
    batch_size=8
)

In [11]:
validation_generator = recognizer.get_batch_generator(
    image_generator=zip(test_images, test_texts),
    batch_size=8
)

In [20]:
recognizer.training_model.compile(optimizer='adam', loss=keras_ocr.tools.ctc_batch_cost)

AttributeError: module 'keras_ocr.tools' has no attribute 'ctc_batch_cost'

In [19]:
# Обучение модели
recognizer.training_model.fit(
    train_generator,
    validation_data=validation_generator,
    steps_per_epoch=len(train_images) // 8,
    validation_steps=len(test_images) // 8,
    epochs=10  # Можно увеличить количество эпох для лучшего результата
)

Epoch 1/10


ValueError: in user code:

    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/tmp/ipykernel_1623/2589838703.py", line 3, in ctc_loss  *
        y_pred = tf.transpose(y_pred, [1, 0, 2])  # Транспонируем предсказания для CTC

    ValueError: Dimension must be 2 but is 3 for '{{node ctc_loss/transpose}} = Transpose[T=DT_FLOAT, Tperm=DT_INT32](model_4/lambda_3/ExpandDims, ctc_loss/transpose/perm)' with input shapes: [?,1], [3].
